# Installations

In [ ]:
install = True
run_test = True

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
install= False
if install == True:
  !pip install scikit-learn==0.24.2

     |████████████████████████████████| 22.3 MB 1.8 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
install = False
if install == True:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config; set_config(display='diagram')

In [ ]:
install = True
if install == True:
  from scipy import stats
  from sklearn.model_selection import train_test_split
  from sklearn.preprocessing import MinMaxScaler, StandardScaler
  from sklearn.pipeline import Pipeline, make_pipeline
  from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
  from sklearn.impute import SimpleImputer, KNNImputer
  from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
  from sklearn.metrics import make_scorer
  from sklearn.model_selection import cross_val_score
  from sklearn.neighbors import KNeighborsRegressor
  from sklearn.linear_model import Ridge, Lasso, LinearRegression
  from sklearn.model_selection import RandomizedSearchCV
  from sklearn.svm import SVR
  from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
  from sklearn.ensemble import AdaBoostRegressor
  from sklearn.tree import DecisionTreeRegressor
  from sklearn.ensemble import VotingRegressor
  from sklearn.ensemble import GradientBoostingRegressor
  from sklearn.ensemble import StackingRegressor
  from sklearn.ensemble import RandomForestRegressor

# Demande du client

> How to predict our CA ?

# Donnée de départs

In [15]:
dataset_d16 = '/content/drive/MyDrive/resto-655-project/resto-project/raw_data/preproc_data_d16.csv'
dataset_d2 = '//content/drive/MyDrive/resto-655-project/resto-project/raw_data/preproc_data_d2.csv'
data_d16 = pd.read_csv(dataset_d16)
data_d2 = pd.read_csv(dataset_d2)
data_d16.head()

,date,jour,service,CA_TTC,CA_HT,TVA,temp,feels_like,temp_min,temp_max,wind_speed,clouds_all,weather_main,weather_description,match_edf,roland_garros,fashion_week,Match Happening,Match Happening-CL,vacances_paris
0,2019-09-01,Dimanche,midi,3548.00,3191.82,356.18,294.08,293.35,292.94,295.87,2.68,0.0,Clear,sky is clear,0.0,0.0,0.0,NaN,NaN,1.0
1,2019-09-01,Dimanche,soir,4577.10,4035.30,541.80,293.37,292.60,291.95,294.40,4.02,0.0,Clear,sky is clear,0.0,0.0,0.0,NaN,NaN,NaN
2,2019-09-02,Lundi,midi,2089.00,1888.79,200.21,294.26,293.47,292.71,295.87,2.60,0.0,Clear,sky is clear,0.0,0.0,0.0,0.0,NaN,0.0
3,2019-09-02,Lundi,soir,5726.50,5056.44,670.06,294.00,293.08,292.25,295.36,0.45,0.0,Clear,sky is clear,0.0,0.0,0.0,NaN,NaN,NaN
4,2019-09-03,Mardi,midi,1745.25,1567.27,177.98,295.07,294.57,293.18,296.42,0.45,40.0,Clouds,scattered clouds,0.0,0.0,0.0,NaN,NaN,0.0


In [18]:
data_d16.fillna(value = "0", inplace = True)
data_d2.fillna(value = "0", inplace = True)
data_d16

,date,jour,service,CA_TTC,CA_HT,TVA,temp,feels_like,temp_min,temp_max,wind_speed,clouds_all,weather_main,weather_description,match_edf,roland_garros,fashion_week,Match Happening,Match Happening-CL,vacances_paris
0,2019-09-01,Dimanche,midi,3548.00,3191.82,356.18,294.08,293.35,292.94,295.87,2.68,0.0,Clear,sky is clear,0.0,0.0,0.0,0,0,1
1,2019-09-01,Dimanche,soir,4577.10,4035.30,541.80,293.37,292.60,291.95,294.40,4.02,0.0,Clear,sky is clear,0.0,0.0,0.0,0,0,0
2,2019-09-02,Lundi,midi,2089.00,1888.79,200.21,294.26,293.47,292.71,295.87,2.60,0.0,Clear,sky is clear,0.0,0.0,0.0,0,0,0
3,2019-09-02,Lundi,soir,5726.50,5056.44,670.06,294.00,293.08,292.25,295.36,0.45,0.0,Clear,sky is clear,0.0,0.0,0.0,0,0,0
4,2019-09-03,Mardi,midi,1745.25,1567.27,177.98,295.07,294.57,293.18,296.42,0.45,40.0,Clouds,scattered clouds,0.0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,2021-07-29,Jeudi,soir,6564.00,5842.95,721.05,295.25,294.82,293.58,296.82,0.89,0.0,Clear,sky is clear,0.0,0.0,0.0,0,0,0
864,2021-07-30,Vendredi,midi,1361.00,1229.24,131.76,295.70,295.29,294.40,297.38,2.24,40.0,Clouds,scattered clouds,0.0,0.0,0.0,0,0,1
865,2021-07-30,Vendredi,soir,7058.00,6263.03,794.97,294.44,294.01,293.27,295.71,1.34,0.0,Clear,sky is clear,0.0,0.0,0.0,0,0,0
866,2021-07-31,Samedi,midi,1655.50,1496.52,158.98,294.69,294.47,294.16,295.59,0.45,0.0,Clear,sky is clear,0.0,0.0,0.0,0,0,1


In [35]:
data_d16.isnull().sum()

date                   0
jour                   0
service                0
CA_TTC                 0
CA_HT                  0
TVA                    0
temp                   0
feels_like             0
temp_min               0
temp_max               0
wind_speed             0
clouds_all             0
weather_main           0
weather_description    0
match_edf              0
roland_garros          0
fashion_week           0
Match Happening        0
Match Happening-CL     0
vacances_paris         0
dtype: int64

In [81]:
weather_main = pd.DataFrame(np.concatenate([data_d16.weather_main.unique(),data_d2.weather_main.unique()])).drop_duplicates()
weather_main.sort_values(by=0,axis=0, inplace=True)
weather_main

,0
0,Clear
1,Clouds
3,Drizzle
5,Fog
7,Haze
4,Mist
2,Rain
13,Snow
6,Thunderstorm


In [80]:
weather_description = pd.DataFrame(np.concatenate([data_d16.weather_description.unique(),data_d2.weather_description.unique()])).drop_duplicates()
weather_description.sort_values(by=0,axis=0, inplace=True)
weather_description

,0
3,broken clouds
7,drizzle
2,few clouds
14,fog
17,haze
13,heavy intensity drizzle
16,heavy intensity rain
10,light intensity drizzle
11,light intensity drizzle rain
9,light intensity shower rain


In [68]:
pd.DataFrame(weather_description).nunique()

0    22
dtype: int64

In [83]:
weather_main

,0
0,Clear
1,Clouds
3,Drizzle
5,Fog
7,Haze
4,Mist
2,Rain
13,Snow
6,Thunderstorm


In [94]:
weather_description.reset_index().drop(columns=['index','level_0'])

,0
0,broken clouds
1,drizzle
2,few clouds
3,fog
4,haze
5,heavy intensity drizzle
6,heavy intensity rain
7,light intensity drizzle
8,light intensity drizzle rain
9,light intensity shower rain


In [103]:
weather_sorted = {
'Clear' : ['sky is clear', np.nan, np.nan,np.nan], 
'Clouds' : ['scattered clouds','few clouds','broken clouds','overcast clouds'],
'Drizzle' : ['light intensity drizzle','drizzle','heavy intensity drizzle',np.nan], 
'Drizzle and Rain' : ['light intensity drizzle rain','rain and drizzle',np.nan,np.nan],
'Fog' : ['fog', np.nan, np.nan,np.nan], 
'Haze' : ['haze', np.nan, np.nan,np.nan], 
'Mist' : ['mist', np.nan, np.nan,np.nan], 
'Rain' : ['light rain','light intensity shower rain','moderate rain', 'heavy intensity rain'], 
'Snow' : ['light snow', np.nan, np.nan,np.nan], 
'Thunderstorm' : ['proximity thunderstorm', 'thunderstorm', 'thunderstorm with light rain','thunderstorm with heavy rain'] 
}

weather_sorted_df = pd.DataFrame(data=weather_sorted)
weather_sorted_df

,Clear,Clouds,Drizzle,Drizzle and Rain,Fog,Haze,Mist,Rain,Snow,Thunderstorm
0,sky is clear,scattered clouds,light intensity drizzle,light intensity drizzle rain,fog,haze,mist,light rain,light snow,proximity thunderstorm
1,NaN,few clouds,drizzle,rain and drizzle,NaN,NaN,NaN,light intensity shower rain,NaN,thunderstorm
2,NaN,broken clouds,heavy intensity drizzle,NaN,NaN,NaN,NaN,moderate rain,NaN,thunderstorm with light rain
3,NaN,overcast clouds,NaN,NaN,NaN,NaN,NaN,heavy intensity rain,NaN,thunderstorm with heavy rain


In [104]:
weather_sorted

{'Clear': ['sky is clear', nan, nan, nan],
 'Clouds': ['scattered clouds',
  'few clouds',
  'broken clouds',
  'overcast clouds'],
 'Drizzle': ['light intensity drizzle',
  'drizzle',
  'heavy intensity drizzle',
  nan],
 'Drizzle and Rain': ['light intensity drizzle rain',
  'rain and drizzle',
  nan,
  nan],
 'Fog': ['fog', nan, nan, nan],
 'Haze': ['haze', nan, nan, nan],
 'Mist': ['mist', nan, nan, nan],
 'Rain': ['light rain',
  'light intensity shower rain',
  'moderate rain',
  'heavy intensity rain'],
 'Snow': ['light snow', nan, nan, nan],
 'Thunderstorm': ['proximity thunderstorm',
  'thunderstorm',
  'thunderstorm with light rain',
  'thunderstorm with heavy rain']}

# Preprocessing

In [20]:
dataTypeSeries = data_d16.dtypes
dataTypeSeries_valuecounts = data_d16.dtypes.value_counts()
print(dataTypeSeries_valuecounts)

float64    12
object      8
dtype: int64


In [26]:
feat_numerical_nunique_d2 = pd.DataFrame(data_d2.select_dtypes(exclude=['object'],include=['int64','float64']).nunique(), columns = ["unique_values"])
feat_numerical_nunique_d2

,unique_values
CA_TTC,1297
CA_HT,1314
TVA,1308
temp,1040
feels_like,1081
temp_min,605
temp_max,676
wind_speed,43
clouds_all,7
match_edf,2


In [27]:
feat_categorical_nunique_d2 = pd.DataFrame(data_d2.select_dtypes(include=['object'],exclude=['int64','float64']).nunique(), columns = ["unique_values"])
feat_categorical_nunique_d2

,unique_values
date,672
jour,7
service,2
weather_main,8
weather_description,21
Match Happening,2
Match Happening-CL,2
vacances_paris,3


In [29]:
feat_numerical_nunique_d16 = pd.DataFrame(data_d16.select_dtypes(exclude=['object'],include=['int64','float64']).nunique(), columns = ["unique_values"])
feat_numerical_nunique_d16

,unique_values
CA_TTC,818
CA_HT,834
TVA,831
temp,705
feels_like,726
temp_min,444
temp_max,489
wind_speed,43
clouds_all,7
match_edf,2


In [30]:
feat_categorical_nunique_d16 = pd.DataFrame(data_d16.select_dtypes(include=['object'],exclude=['int64','float64']).nunique(), columns = ["unique_values"])
feat_categorical_nunique_d16

,unique_values
date,427
jour,7
service,2
weather_main,8
weather_description,19
Match Happening,2
Match Happening-CL,2
vacances_paris,3


In [ ]:
data.select_dtypes(include=['object'],exclude=['int64','float64']).nunique().index.sort_values

# Model

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
cv_results = cross_validate(model, X_train, y_train, cv = 20)
cv_score = cv_results['test_score'].mean()
cv_score

In [ ]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', MinMaxScaler()),
    ('model', LinearRegression())
])

In [ ]:
pipe.get_params()

In [ ]:
from sklearn.model_selection import GridSearchCV
run_test = False
if run_test == True:
  grid_search = GridSearchCV(
      pipe, 
      param_grid={'imputer__n_neighbors': [4,5,6]
      },
      cv=5,
      scoring="accuracy")

  grid_search.fit(X, y)
  grid_search.best_params_

In [ ]:
run_test = False
if run_test == True:
  tuned_pipe = grid_search.best_estimator_
  tuned_pipe

In [ ]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(tuned_pipe, X,y, cv = 20, scoring = "accuracy")

base_score = cv_results['test_score'].mean()

base_score